# Milestone 1, 2 & 3

In [6]:
import re
import math

### Tokenizer

In [7]:
class Tokenizer:
    def __init__(self):
        self.patterns = [
            (r'\d+\.\d+', lambda x: float(x)),  # Floating-point numbers
            (r'\d+', lambda x: int(x)),          # Integers
            (r'\+', lambda x: x),
            (r'-', lambda x: x),
            (r'\*', lambda x: x),
            (r'/', lambda x: x),
            (r'\(', lambda x: x),
            (r'\)', lambda x: x),
            (r'sqrt', lambda x: x),
            (r'sin', lambda x: x),
            (r'cos', lambda x: x),
            (r'tan', lambda x: x),
            (r'pi', lambda x: math.pi),
        ]

    def tokenize(self, expression):
        tokens = []
        while expression:
            for pattern, handler in self.patterns:
                match = re.match(pattern, expression)
                if match:
                    token = match.group(0)
                    tokens.append(handler(token))
                    expression = expression[len(token):].strip()
                    break
            else:
                raise ValueError("Invalid character in input")
        return tokens

### Parser

In [8]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token_index = 0

    def parse(self):
        try:
            result = self.expression()
            if self.current_token_index != len(self.tokens):
                raise SyntaxError("Invalid syntax")
            return result
        except ZeroDivisionError:
            raise ZeroDivisionError("Division by zero")

    def match(self, expected_token):
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index] == expected_token:
            self.current_token_index += 1
            return True
        else:
            return False

    def expression(self):
        return self.addition()

    def addition(self):
        result = self.multiplication()

        while self.match('+') or self.match('-'):
            operator = self.tokens[self.current_token_index - 1]
            right_operand = self.multiplication()
            if operator == '+':
                result += right_operand
            else:
                result -= right_operand

        return result

    def multiplication(self):
        result = self.primary()

        while self.match('*') or self.match('/'):
            operator = self.tokens[self.current_token_index - 1]
            right_operand = self.primary()
            if operator == '*':
                result *= right_operand
            else:
                if right_operand == 0:
                    raise ZeroDivisionError("Division by zero")
                result /= right_operand

        return result

    def primary(self):
        if self.match('('):
            result = self.expression()
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return result
        elif isinstance(self.tokens[self.current_token_index], int) or isinstance(self.tokens[self.current_token_index], float):
            result = self.tokens[self.current_token_index]
            self.current_token_index += 1  # Move to the next token
            return result
        elif self.match('sqrt'):
            if not self.match('('):
                raise SyntaxError("Invalid syntax")
            result = self.expression()
            if result < 0:
                raise ValueError("Square root of negative number")
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return math.sqrt(result)
        elif self.match('sin'):
            if not self.match('('):
                raise SyntaxError("Invalid syntax")
            result = self.expression()
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return math.sin(result)
        elif self.match('cos'):
            if not self.match('('):
                raise SyntaxError("Invalid syntax")
            result = self.expression()
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return math.cos(result)
        elif self.match('tan'):
            if not self.match('('):
                raise SyntaxError("Invalid syntax")
            result = self.expression()
            if not self.match(')'):
                raise SyntaxError("Invalid syntax")
            return math.tan(result)
        else:
            raise SyntaxError("Invalid syntax")

In [ ]:
# # Test the Parser
# def test_parser():
#     expression = input("Enter an arithmetic expression: ")
#     tokenizer = Tokenizer()
#     tokens = tokenizer.tokenize(expression)
#     if tokens is not None:
#         print("Tokens:", tokens)  # Print tokens for debugging
#         parser = Parser(tokens)
#         try:
#             result = parser.parse()
#             print("Result:", result)
#         except (SyntaxError, ZeroDivisionError, ValueError) as e:
#             print("Error:", e)

# test_parser()

### GUI

In [48]:
import tkinter as tk
from tkinter import ttk, messagebox

class CalculatorApp:
    def __init__(self, master):
        self.master = master
        master.title("Calculator")

        # Create a style object
        self.style = ttk.Style()
        
        # Configure style for various elements
        self.style.configure('TButton', font=('Helvetica', 12), foreground='#000000', background='#4CAF50', padding=10, borderwidth=0)
        self.style.map('TButton', background=[('active', '#000000')])  # Change button color on hover
        
        self.style.configure('TLabel', font=('Helvetica', 12), foreground='#333333', background='#FFFFFF', padding=10)
        
        self.style.configure('TText', font=('Helvetica', 14), foreground='#333333', background='#f0f0f0', borderwidth=0, padx=10, pady=10)
        
        # Textbox for input expression
        self.textbox = tk.Text(master, height=10, width=50)
        self.textbox.pack(pady=10, padx=10, fill='both', expand=True)

        # Button to check syntax
        self.check_button = ttk.Button(master, text="Step 1 - Check Syntax", command=self.check_syntax)
        self.check_button.pack(pady=5, padx=10, fill='x')

        # Button to execute code
        self.execute_button = ttk.Button(master, text="Step 2 - Execute", command=self.execute_code, state="disabled")
        self.execute_button.pack(pady=5, padx=10, fill='x')

        # Error logger label
        self.error_logger = ttk.Label(master, text="", foreground="red")
        self.error_logger.pack(pady=5, padx=10, fill='x')

        # Tokens label
        self.tokens_label = ttk.Label(master, text="")
        self.tokens_label.pack(pady=5, padx=10, fill='x')

    def check_syntax(self):
        code = self.textbox.get("1.0", "end-1c")
        tokenizer = Tokenizer()
        try:
            tokens = tokenizer.tokenize(code)
            parser = Parser(tokens)
            parser.parse()
            self.error_logger.config(text="Syntax is correct!", foreground="green")
            self.tokens_label.config(text="Tokens: " + str(tokens), foreground="black")
            self.execute_button.config(state="normal")  # Enable the Execute button
        except Exception as e:
            self.error_logger.config(text=str(e), foreground="red")
            self.tokens_label.config(text="")
            self.execute_button.config(state="disabled")  # Disable the Execute button


    def execute_code(self):
        code = self.textbox.get("1.0", "end-1c")
        tokenizer = Tokenizer()
        try:
            tokens = tokenizer.tokenize(code)
            parser = Parser(tokens)
            result = parser.parse()
            messagebox.showinfo("Result", f"The result is: {result}")
        except Exception as e:
            messagebox.showerror("Error", str(e))

root = tk.Tk()
app = CalculatorApp(root)
root.mainloop()
